In [8]:
import os
import requests
import zipfile
from clearbg import logger

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir: Path
    train_source_url: str
    test_source_url: str
    train_local_zipped_path: Path
    test_local_zipped_path: Path

In [10]:
from clearbg.constants import *
from clearbg.utils.common import read_yaml, create_directories

In [11]:
print(CONFIG_FILE_PATH.absolute())

C:\dev\ml\projects\MLOPS_Project\config\config.yaml


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            train_source_url=config.train_source_url,
            test_source_url=config.test_source_url,
            train_local_zipped_path=config.train_local_zipped_path,
            test_local_zipped_path=config.test_local_zipped_path
        )
        return data_ingestion_config

In [13]:
class DataIngestion:
    def __init__(self, config=DataIngestionConfig):
        self.config = config

    def download_train_data(self) -> str:
        """
        Download Train Dataset
        """
        try:
            train_source_url = self.config.train_source_url
            logger.info(f"Downloading TRAIN DATASET from: {train_source_url}")
            
            response = requests.get(train_source_url)
            response.raise_for_status() 

            root_dir = self.config.root_dir
            os.makedirs(root_dir, exist_ok=True)
            filename = os.path.basename(train_source_url)
            file_path = os.path.join(root_dir, filename)
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            logger.info(f"Downloaded TRAIN DATASET: {file_path}")
            return file_path 
            
        except Exception as e:
            logger.error(f"Failed to download train dataset: {e}")
            raise e
    
    def download_test_data(self) -> str:
        """
        Download Test Dataset
        """
        try:
            test_source_url = self.config.test_source_url
            logger.info(f"Downloading TEST DATASET from: {test_source_url}")

            response = requests.get(test_source_url)
            response.raise_for_status()

            root_dir = self.config.root_dir
            os.makedirs(root_dir, exist_ok=True)
            filename = os.path.basename(test_source_url)
            file_path = os.path.join(root_dir, filename)
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            logger.info(f"Downloaded TEST DATASET: {file_path}")
            
        except Exception as e:
            logger.error(f"Failed to download test dataset: {e}")
            raise e
    
    def download_data(self, mode="all") -> str:
        """
        Download Train and Test Datasets
        """
        try:
            if mode == "train":
                return self.download_train_data()
            elif mode == "test":
                return self.download_test_data()
            elif mode == "all":
                train_path = self.download_train_data()
                test_path = self.download_test_data()

        except Exception as e:
            logger.error(f"Failed to download datasets: {e}")
            raise e 

    def extract_train_data(self) -> str:
        """
        Extract Zipped Train Dataset
        """
        try:
            root_dir = self.config.root_dir
            train_local_zipped_path = self.config.train_local_zipped_path
            
            os.makedirs(root_dir, exist_ok=True)

            logger.info(f"Extracting TRAIN DATASET from: {train_local_zipped_path}")
            with zipfile.ZipFile(train_local_zipped_path, 'r') as zip_ref:
                zip_ref.extractall(root_dir)
            logger.info(f"Extracted TRAIN DATASET to: {root_dir}")
            

        except Exception as e:
            logger.error(f"Failed to extract train dataset: {e}")
            raise e

    def extract_test_data(self) -> str:
        """
        Extract Zipped Test Dataset
        """
        try:
            root_dir = self.config.root_dir
            test_local_zipped_path = self.config.test_local_zipped_path
            
            os.makedirs(root_dir, exist_ok=True)

            logger.info(f"Extracting TEST DATASET from: {test_local_zipped_path}")
            with zipfile.ZipFile(test_local_zipped_path, 'r') as zip_ref:
                zip_ref.extractall(root_dir)
            logger.info(f"Extracted TEST DATASET to: {root_dir}")
            
        except Exception as e:
            logger.error(f"Failed to extract test dataset: {e}")
            raise e

    def extract_data(self, mode="all") -> str:
        """
        Extract Train and Test Datasets
        """
        try:
            if mode == "train":
                return self.extract_train_data()
            elif mode == "test":
                return self.extract_test_data()
            elif mode == "all":
                train_path = self.extract_train_data()
                test_path = self.extract_test_data()

        except Exception as e:
            logger.error(f"Failed to extract datasets: {e}")
            raise e

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data(mode="train")
    data_ingestion.extract_data(mode="train")
except Exception as e:
    raise e
    

[2024-10-10 15:50:42,430: INFO: common: yaml file: C:\dev\ml\projects\MLOPS_Project\config\config.yaml loaded successfully]
[2024-10-10 15:50:42,432: INFO: common: yaml file: C:\dev\ml\projects\MLOPS_Project\params.yaml loaded successfully]
[2024-10-10 15:50:42,433: INFO: common: created directory at: artifacts]
[2024-10-10 15:50:42,434: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-10 15:50:42,435: INFO: 2476120642: Downloading TRAIN DATASET from: http://saliencydetection.net/duts/download/DUTS-TR.zip]
[2024-10-10 15:55:50,034: INFO: 2476120642: Downloaded TRAIN DATASET: artifacts/data_ingestion\DUTS-TR.zip]
[2024-10-10 15:55:50,111: INFO: 2476120642: Extracting TRAIN DATASET from: artifacts/data_ingestion/DUTS-TR.zip]
[2024-10-10 15:56:24,463: INFO: 2476120642: Extracted TRAIN DATASET to: artifacts/data_ingestion]
